In [179]:
import torch
from dataclasses import dataclass

@dataclass
class TorchLogisticRegressor:
    lr: float = 0.1
    l1_w: float = 0.0
    l2_w: float = 1.0
    max_iterations: int = 1000
    tolerance: float = 1e-4
    intercept: bool = True
    device: str = 'cuda:0'

    def fit(self, x, y):
        if y.ndim == 1:
            num_classes = len(np.unique(y))
        else:
            num_classes = y.shape[-1]
        #x = (x - np.mean(x, axis=0, keepdims=True))/(np.std(x, axis=0, keepdims=True)+1e-6)
        self.model = torch.nn.Linear(x.shape[-1], num_classes, bias=self.intercept, dtype=torch.float64, device=self.device)
        optimizer = torch.optim.LBFGS(self.model.parameters(), 
                                      lr=self.lr, 
                                      history_size=100, 
                                      max_iter=20, 
                                      line_search_fn='strong_wolfe')
        criterion = torch.nn.CrossEntropyLoss()

        def closure():
            optimizer.zero_grad()
            x_ = torch.from_numpy(x).to(dtype=torch.float64, device=self.device)
            y_ = torch.from_numpy(y).to(device=self.device)
            outputs = self.model(x_)
            loss = criterion(outputs, y_)

            l1_penalty = sum(p.abs().sum() for p in self.model.parameters())/x_.shape[-1]
            l2_penalty = sum(p.pow(2.0).sum() for p in self.model.parameters())/x_.shape[-1]
            loss += self.l1_w * l1_penalty + self.l2_w * l2_penalty  # Apply regularization
            
            loss.backward()
            return loss

        last_loss = np.inf
        i = 0
        while i < self.max_iterations:
            loss = optimizer.step(closure)
            if (last_loss - loss) < self.tolerance:
                break
            i+=1
            last_loss = loss
        if i == self.max_iterations:
            print(f'Warning: model did not converge after running {self.max_iterations} iterations. Consider increasing max_iterations.')

    def predict(self, x):
        return self.model(torch.from_numpy(x).to(dtype=torch.float64, device=self.device)).detach().cpu().numpy()

In [180]:
import time

import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype('int')

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000
)

scaler = StandardScaler()
X_train = X_train / np.max(X_train)
X_test = X_test / np.max(X_test)

model_torch = TorchLogisticRegressor(max_iterations=1000, tolerance = 1e-4, l2_w=1.0, lr=1.0)
model_sklearn = LogisticRegression(max_iter=1000, C=1)

model_torch.fit(X_train, y_train)
model_sklearn.fit(X_train, y_train)

pred_torch = model_torch.predict(X_test)
pred_sklearn = model_sklearn.predict(X_test)

print('Torch: {}'.format((np.argmax(pred_torch,axis=-1) == y_test).mean()))
print('Sklearn: {}'.format((pred_sklearn == y_test).mean()))

Torch: 0.9013
Sklearn: 0.8895
